## Introdução

As estruturas de códgios a seguir foram criadas para testar uma estratégia que se baseia no cálculo do valor esperado de um ativo pela clássica fórmula do CAPM.
Após chegar no valor esperado do ativo em questão, ele é comparado ao valor real que ele atingiu. Caso o valor real seja maior do que se era esperado, significa que naquele momento ele está caro, portanto, recomenda-se não comprá-lo. Em compensação, caso o valor real seja menor do que se era esperado, significa que ele está barato e, consequentemente, possui sinal verde para a compra do mesmo.

Muito importante enfatizar que essa estratégia visa principalmente a diminuição de perdas e não a maximização dos retornos, ou seja, se uma certa carteira tiver performance negativa, ao aplicar essa estratégia, e selecionar apenas algumas ações desta carteira, essas ações selecionadas darão perdas menores do que se considerasse a carteira toda.

## Pacotes

In [292]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime

## Extração de dados

In [909]:
#funcao que extrai dados de acoes de uma base de dados propria
def query_ticker(lista, data_inicial, data_final, senha, user, host):    
    engine = create_engine(f'postgresql://{user}:{senha}@{host}:5432/')
    
    df = pd.DataFrame()
    
    ibov = pd.read_sql_query(f"""select data, preco from índices_diários where data between '{data_inicial}' and '{data_final}' and nome in ('IBOV')order by data ASC""" , con=engine)
    
    df['IBOV'] = ibov['preco']
    df.index = ibov['data']
    df = df.iloc[1: , :]
    
    for ticker in lista:
        df0 = pd.read_sql_query(f"""select * from dados_diários where data between '{data_inicial}' and '{data_final}' and nome in ('{ticker}') order by data ASC""" , con=engine)
        nome = df0['nome'][1]
        df0 = df0[['fechamento']]
        try:
            df0 = df0.pct_change().dropna()
            df[nome] = df0['fechamento'].values
        except:
            row = len(df) - len(df0)
            df_na = pd.DataFrame(index=np.arange(row), columns=np.arange(1))
            df0 = pd.concat([df_na, df0]).reset_index(drop = True)
            df[nome] = df0['fechamento'].values
            
                

    return df





As ações escolhidas a seguir foram selecionadas a partir da cateira recomendada da XP de agosto de 2021.

In [629]:
lista = ["B3SA3","SULA11","ARZZ3","RENT3","KLBN11","MULT3","RDOR3","VALE3","LAME4","ASAI3"]

In [771]:
df = query_ticker(lista,'01/01/2019','09/14/2021', 'password', 'user', 'host')

In [885]:
df.tail()

,IBOV,B3SA3,SULA11,ARZZ3,RENT3,KLBN11,MULT3,RDOR3,VALE3,LAME4,ASAI3
data,,,,,,,,,,,
2021-09-08,-0.037805,-0.083799,-0.037396,-0.046998,0.080281,-0.011267,-0.043817,-0.018507,-0.020812,-0.085484,-0.023978
2021-09-09,0.017176,0.038872,0.039928,0.009451,0.002004,0.019646,0.043279,0.011571,-0.003577,0.022928,0.016192
2021-09-10,-0.009319,0.013206,-0.009339,-0.001801,-0.045500,-0.021965,-0.014641,-0.024811,0.001162,-0.027586,-0.001099
2021-09-13,0.018533,0.028240,0.032821,0.011304,0.030208,0.008668,0.027241,0.036765,-0.000527,0.072695,0.026953
2021-09-14,-0.001916,-0.004930,0.002705,0.006897,-0.016949,0.005859,-0.012054,-0.002837,-0.007071,-0.016529,-0.018211


Antes de irmos para os cálculos, essa é a fórmula para calcular o retorno esperado:

<img src = "https://empreenderdinheiro.com.br/wp-content/uploads/2020/02/capm_formula_1.png" style = "height:150px; width:500px" />


Para fins mais práticos, descondieraremos o risk free rate da fórmula, porque, por se tratar de dados diários, no final ele não fará tanta diferença.

Então, para chegar no retorno esperado, faltam duas variáveis da fórmula: o beta e o retorno do mercado

## Cálculo BETA

In [886]:
df = df.dropna()

In [887]:
matrix_covariance = df[:-1].cov()

In [888]:
matrix_covariance

,IBOV,B3SA3,SULA11,ARZZ3,RENT3,KLBN11,MULT3,RDOR3,VALE3,LAME4,ASAI3
IBOV,0.000134,0.000170,0.000155,0.000133,0.000158,0.000051,0.000180,0.000066,0.000106,0.000205,0.000106
B3SA3,0.000170,0.000514,0.000227,0.000162,0.000164,0.000047,0.000238,0.000039,0.000074,0.000209,0.000169
SULA11,0.000155,0.000227,0.000555,0.000188,0.000195,0.000072,0.000270,0.000094,0.000017,0.000318,0.000112
ARZZ3,0.000133,0.000162,0.000188,0.000390,0.000161,0.000066,0.000202,0.000112,0.000076,0.000248,0.000160
RENT3,0.000158,0.000164,0.000195,0.000161,0.000565,0.000024,0.000274,0.000178,0.000076,0.000268,0.000165
KLBN11,0.000051,0.000047,0.000072,0.000066,0.000024,0.000266,0.000034,0.000019,0.000077,0.000016,0.000057
MULT3,0.000180,0.000238,0.000270,0.000202,0.000274,0.000034,0.000528,0.000103,0.000072,0.000302,0.000145
RDOR3,0.000066,0.000039,0.000094,0.000112,0.000178,0.000019,0.000103,0.000416,0.000005,0.000153,0.000072
VALE3,0.000106,0.000074,0.000017,0.000076,0.000076,0.000077,0.000072,0.000005,0.000377,0.000052,0.000062
LAME4,0.000205,0.000209,0.000318,0.000248,0.000268,0.000016,0.000302,0.000153,0.000052,0.001027,0.000151


In [889]:
covariance = np.array(matrix_covariance['IBOV'])
covariance

array([1.34217093e-04, 1.69747611e-04, 1.54598416e-04, 1.33125717e-04,
       1.58109938e-04, 5.08905590e-05, 1.79610160e-04, 6.56720456e-05,
       1.05883516e-04, 2.04810343e-04, 1.05538311e-04])

In [890]:
variance = (df[:-1].var())[0]
variance

0.00013421709348387023

In [891]:
beta = covariance/variance
beta = np.delete(beta, 0)
beta

array([1.26472424, 1.15185341, 0.99186857, 1.1780164 , 0.379166  ,
       1.3382063 , 0.48929718, 0.7888974 , 1.52596318, 0.78632541])

In [892]:
df_beta = pd.DataFrame({'Ticker':lista,
                      'Beta':beta})

df_beta

,Ticker,Beta
0,B3SA3,1.264724
1,SULA11,1.151853
2,ARZZ3,0.991869
3,RENT3,1.178016
4,KLBN11,0.379166
5,MULT3,1.338206
6,RDOR3,0.489297
7,VALE3,0.788897
8,LAME4,1.525963
9,ASAI3,0.786325


## Retorno Esperado

Após calcular o beta, basta pegar o retorno de mercado. Nesse caso, estamos querendo chegar no retorno esperado do dia 13/09, então pegamos o retorno do ibovespa neste exato dia e calculamos o retorno esperado apenas multiplicando-se o beta pelo retorno do mercado, já que desconsideramos o risk free.

In [893]:
a = df[-2:-1].drop('IBOV', axis=1).T.values
b = a.ravel()

retorno_esperado = pd.DataFrame({'Ticker':lista,
                                'Retorno Esperado':df_beta['Beta']*list(df['IBOV'][-2:-1]),
                                 'Retorno Real':b
                                })

retorno_esperado

,Ticker,Retorno Esperado,Retorno Real
0,B3SA3,0.023439,0.028240
1,SULA11,0.021347,0.032821
2,ARZZ3,0.018382,0.011304
3,RENT3,0.021832,0.030208
4,KLBN11,0.007027,0.008668
5,MULT3,0.024800,0.027241
6,RDOR3,0.009068,0.036765
7,VALE3,0.014620,-0.000527
8,LAME4,0.028280,0.072695
9,ASAI3,0.014573,0.026953


## Comprando Ações Baratas

In [895]:
status = ['BARATO' if retorno_esperado['Retorno Real'][i]<retorno_esperado['Retorno Esperado'][i] else 'CARO' for i in range(len(retorno_esperado))]


In [896]:
retorno_esperado['Status'] = status
retorno_esperado

,Ticker,Retorno Esperado,Retorno Real,Status
0,B3SA3,0.023439,0.028240,CARO
1,SULA11,0.021347,0.032821,CARO
2,ARZZ3,0.018382,0.011304,BARATO
3,RENT3,0.021832,0.030208,CARO
4,KLBN11,0.007027,0.008668,CARO
5,MULT3,0.024800,0.027241,CARO
6,RDOR3,0.009068,0.036765,CARO
7,VALE3,0.014620,-0.000527,BARATO
8,LAME4,0.028280,0.072695,CARO
9,ASAI3,0.014573,0.026953,CARO


## Resultados Finais

Por último, como temos já quais são as ações baratas e as caras, basta pegar a média dos retornos doa dia 14/09 para checar se obedecendo a esses critérios de barato e caro, quanto que teria performado só comprando as caras, só as baratas e a carteira recomendada pela XP sem nenhum filtro.

In [904]:
retorno_mercado = np.mean(df.loc[:, df.columns != 'IBOV'].iloc[-1])
retorno_mercado

-0.00631201704665183

In [899]:
baratas = [retorno_esperado.iloc[i,0] for i in range(len(retorno_esperado))  if retorno_esperado.iloc[i,3] == 'BARATO']
caras = [retorno_esperado.iloc[i,0] for i in range(len(retorno_esperado))  if retorno_esperado.iloc[i,3] == 'CARO']

In [906]:
retorno_baratas = np.mean(df[baratas].iloc[-1])
retorno_caras = np.mean(df[caras].iloc[-1])

In [908]:
resultados_finais = pd.DataFrame({'Mercado':[retorno_mercado], 
                                 'Baratas':[retorno_baratas],
                                 'Caras':[retorno_caras]})

resultados_finais

,Mercado,Baratas,Caras
0,-0.006312,-0.000087,-0.007868


## OBS
Vale destacar que, para estudos mais precisos, seria necessário realizar o backtest desta estratégia por períodos mais longos, utiliar uma carteira com um maior número de ativos e seria interessante explorar alguma outra abordagem para mudar o escopo e maximizar os retornos, ao  invés de apenas minimizar as perdas.